# Carregando Bibliotecas

In [ ]:
import pandas as pd
from pathlib import Path
from mysql import connector as cn #pip install mysql-connector-python
import datetime 
from rich.jupyter import print


#Conection Data:
CON_DATA = {
       'user':'db_med',
       'password': 'db_med',
       'host': '127.0.0.1',
       'database' : 'db_med'
       }

#Pasta de Origem
PATH_TO_TESTE = '/home/bruno/Documentos/Python/projetos_python/Estudos/XP_MOD5/BootCampArquBigData/Mod02/trabalho_pratic/src/extracted_data/2020-01-01.csv'

PATH = '/home/bruno/Documentos/Python/projetos_python/estudos/XP_MOD5/BootCampArquBigData/Desafio/src/processamento_licencas_medicas'

# Lendo Arquivos, Transformando em DataFrame

In [ ]:
print('Iniciando a coleta de dados de Licenças Medicas...', end='')

arquivos_lidos = []
for file in Path(PATH).iterdir():
    if file.name.endswith('csv'):
        # print(f'+ {file.name} aos dados...')
        arquivos_lidos.append(pd.read_csv(Path(file).as_posix(),sep='|', encoding= 'ISO-8859-1'))
print('[green]OK')
print('Criando DF...', end='')

df = pd.concat(arquivos_lidos)
del arquivos_lidos
print('[green]OK')



In [ ]:
df.info()

In [ ]:
df.describe()

# Tratando Dados

## Preenchendo dados Faltantes

### Salario

In [ ]:
print(f" Seram substituidas {df['salario'].isnull().sum()} observacoes...")
df['salario'] = df['salario'].fillna(df['salario'].mean())


### QTD Filhos

In [ ]:
print(f" Seram substituidas {df['qtd_filhos'].isnull().sum()} observacoes...")
df['qtd_filhos'] = df['qtd_filhos'].fillna(
    round(df['qtd_filhos'].mean())
    )


## Resetando o Index, Eliminando Duplicados e Nulos/Vazios

In [ ]:
df = df.reset_index()
df.drop(['index'], axis=1, inplace=True)
print(f'Seram [red]eliminanods {df.duplicated().sum()}[/red] dados duplicados.')
df.drop_duplicates(inplace=True)
print(f'Seram [red]eliminados {sum(df.isna().sum().values)}[/red] dados nulos/ausentes')
df.dropna(inplace=True)



# Banco de Dados

## Conectando ao Banco de Dados

In [ ]:
cnx = cn.connect(**CON_DATA) # tambem se pode utilizar com o with para evitar problemas
# with cn.connect(**CON_DATA) as cnx:
    # pass 

if cnx.is_connected():
    print('Conexão Realizada com sucesso!')

## Definindo Funcoes

In [ ]:
def inserir_df_em_db(conector,tabela_destino,df:pd.DataFrame):
    colunas = '`' + '`, `'.join(df.columns.values) + '`'
    for i in range(df.count().values[0]):
        conteudo = "'" + "', '".join([str(item) for item in df.loc[i].values]) + "'"

        texto = f'Inserindo linha {i} : {conteudo}... '
        
        with conector.cursor() as cursor:
            comando = f'''
            INSERT INTO {tabela_destino} ({colunas}) VALUES ({conteudo});
            '''
            try:

                cursor.execute(comando)
                conector.commit() #confirmando as alteracoes
                cursor.close()

            except:

                print(f'{texto}[red]Erro')
                
def get_data_from_table(conector, nome_tabela):
    dicionario = dict()
    with conector.cursor() as cursor:
        result = cursor.execute(F'''SELECT * FROM {nome_tabela};''')
        rows = cursor.fetchall()
        for key, value in rows:
            dicionario[value] = key
        cursor.close()
    return dicionario



## Inserindo Dados Primarios no DB

### Estados

#### Inserindo no Banco de Dados

In [ ]:
df_aux = pd.DataFrame()
df_aux['estado'] = df['estado_colaborador'].unique()
inserir_df_em_db(cnx, 'tb_estado', df_aux)

#### Inserindo no cod_estado no DF

In [ ]:
dict_encode = get_data_from_table(cnx,'tb_estado')
df['cod_estado'] = df['estado_colaborador'].map(dict_encode)

### Estado-Civil

#### Inserindo no Banco de Dados

In [ ]:
df_aux = pd.DataFrame()
df_aux['estado_civil'] = df['estado_civil'].unique()
inserir_df_em_db(cnx, 'tb_estado_civil', df_aux)


#### Inserindo no cod_estado_civil no DF

In [ ]:
dict_encode = get_data_from_table(cnx,'tb_estado_civil')
df['cod_estado_civil'] = df['estado_civil'].map(dict_encode)

### Escolaridade

#### Inserindo no Banco de Dados

In [ ]:
df_aux = pd.DataFrame()
df_aux['escolaridade'] = df['escolaridade'].unique()
inserir_df_em_db(cnx, 'tb_escolaridade', df_aux)

#### Inserindo no cod_escolaridade no DF

In [ ]:
dict_encode = get_data_from_table(cnx,'tb_escolaridade')
df['cod_escolaridade'] = df['escolaridade'].map(dict_encode)

### Hobbie

#### Inserindo no Banco de Dados

In [ ]:
df_aux = pd.DataFrame()
df_aux['hobbie'] = df['hobbie'].unique()
inserir_df_em_db(cnx, 'tb_hobbie', df_aux)

#### Inserindo no cod_hobbie no DF

In [ ]:
dict_encode = get_data_from_table(cnx,'tb_hobbie')
df['cod_hobbie'] = df['hobbie'].map(dict_encode)

### Especialidade

#### Inserindo no Banco de Dados


In [ ]:
df_aux = pd.DataFrame()
df_aux['especialidade'] = df['especialidade'].unique()
inserir_df_em_db(cnx, 'tb_especialidade', df_aux)

#### Inserindo no cod_especialidade no DF


In [ ]:
dict_encode = get_data_from_table(cnx,'tb_especialidade')
df['cod_especialidade'] = df['especialidade'].map(dict_encode)

### Motivo Licensa

#### Inserindo no Banco de Dados


In [ ]:
df_aux = pd.DataFrame()
df_aux['motivo_licenca'] = df['motivo_licenca'].unique()
inserir_df_em_db(cnx, 'tb_motivo_licenca', df_aux)

#### Inserindo no cod_licenca no DF


In [ ]:
dict_encode = get_data_from_table(cnx,'tb_motivo_licenca')
df['cod_motivo_licenca'] = df['motivo_licenca'].map(dict_encode)


## Dados Secundarios

### Medico

#### Inserindo no Banco de Dados


In [ ]:
df_aux = pd.DataFrame()
df_aux = df[['nome_medico','cod_especialidade']].drop_duplicates()
df_aux = df_aux.reset_index()
df_aux.drop(['index'], axis=1, inplace=True)
inserir_df_em_db(cnx, 'tb_medico', df_aux)

#### Inserindo no cod_medico no DF


In [ ]:
dict_encode = dict()

with cnx.cursor() as cursor:
        result = cursor.execute(F'''SELECT cod_medico, nome_medico FROM {'tb_medico'};''')
        rows = cursor.fetchall()
        for key, value in rows:
            dict_encode[value] = key
df['cod_medico'] = df['nome_medico'].map(dict_encode)

### Colaborador

#### Inserindo no Banco de Dados

In [ ]:
df_aux = pd.DataFrame()
df_aux = df[['nome_colaborador','data_nascimento',
            'sexo_colaborador',
            'salario',
            'qtd_filhos',
            'possui_pet',
            'cod_estado_civil',
            'cod_escolaridade',
            'cod_hobbie',
            'cod_estado']].drop_duplicates()
#remover nomes  duplicados
df_aux[ df_aux['nome_colaborador'].duplicated().map(lambda x : not x)].count()

df_aux = df_aux.reset_index()
df_aux.drop(['index'], axis=1, inplace=True)
inserir_df_em_db(cnx, 'tb_colaborador', df_aux)


#### Inserindo cod_colaborador no DF


In [ ]:
dict_encode = dict()

with cnx.cursor() as cursor:
        result = cursor.execute(F'''SELECT cod_colaborador, nome_colaborador FROM {'tb_colaborador'};''')
        rows = cursor.fetchall()
        for key, value in rows:
            dict_encode[value] = key
df['cod_colaborador'] = df['nome_colaborador'].map(dict_encode)

### Processamento Licenca

#### Inserindo no Banco de Dados

In [118]:
# como n tem uma chave unica que limiete os dados como um documetno ou algo 
# deve-se ter cuidado pra n dublicar os dados
dados = False
if dados:
    df_aux = pd.DataFrame()
    df_aux = df[['data_processamento','inicio_licenca',
                'fim_licenca',
                'duracao_licenca',
                'cod_colaborador',
                'cod_medico',
                'cod_motivo_licenca']]
    #resetar index por precaucao
    df_aux = df_aux.reset_index()
    df_aux.drop(['index'], axis=1, inplace=True)
    inserir_df_em_db(cnx, 'tb_processamento_licenca', df_aux)